# `OPENJSON` and `JSON_QUERY`

`OPENJSON` is _not_ among the <acronym title="JavaScript Object Notation">JSON</acronym> functions [📖 [docs](https://learn.microsoft.com/en-us/sql/t-sql/functions/json-functions-transact-sql?view=sql-server-ver16)]. I must remind myself that [the docs clearly state](https://learn.microsoft.com/en-us/sql/t-sql/functions/openjson-transact-sql?view=sql-server-ver16) that `OPENJSON` is a “table-valued function.” This understanding of `OPENJSON` is useful to have while working with `JSON_QUERY` [📖 [docs](https://learn.microsoft.com/en-us/sql/t-sql/functions/json-query-transact-sql?view=sql-server-ver16)].

## `JSON_QUERY` can only access arrays by index

`JSON_QUERY` can only access arrays by index but because of `OPEN_JSON` this is (probably) not considered a limitation (by the SQL Server designers at Microsoft). This means we can start with a JSON string like this:

In [1]:
EXEC sys.sp_set_session_context
    @key = N'myJson',
    @value = '
{
    "Orders": [
        {
            "OrderNumber":"SO43659",
            "OrderDate":"2011-05-31T00:00:00",
            "AccountNumber":"AW29825",
            "ItemPrice":2024.9940,
            "ItemQuantity":1
        },
        {
            "OrderNumber":"SO43661",
            "OrderDate":"2011-06-01T00:00:00",
            "AccountNumber":"AW73565",
            "ItemPrice":2024.9940,
            "ItemQuantity":3
        }
    ]
}'

Commands completed successfully.

Total execution time: 00:00:00.001

When for some reason we need to see the _second_ item in the `Orders` array, `JSON_QUERY` can handle this:

In [2]:
DECLARE @myJson NVARCHAR(MAX) = CONVERT(NVARCHAR(MAX), SESSION_CONTEXT(N'myJson'))

SELECT
    [SECOND_ORDER] = JSON_QUERY(@myJson, '$.Orders[1]')
,   [SECOND_ORDER_NUMBER?] = JSON_QUERY(@myJson, '$.Orders[1].OrderNumber')

(1 row affected)

Total execution time: 00:00:00.002

SECOND_ORDER,SECOND_ORDER_NUMBER?
"{ ""OrderNumber"":""SO43661"", ""OrderDate"":""2011-06-01T00:00:00"", ""AccountNumber"":""AW73565"", ""ItemPrice"":2024.9940, ""ItemQuantity"":3 }",NULL


The projection of `[SECOND_ORDER]` faithfully demonstrates that a JSON object can be extracted with `JSON_QUERY`. However, the `NULL` value of `[SECOND_ORDER_NUMBER?]` shows us how quickly `JSON_QUERY` can fall apart. With glaring omission and copious silence, it is clear that the syntax of `JSON_QUERY` is not like that of Oracle [📖 [docs](https://docs.oracle.com/en/database/oracle/oracle-database/19/sqlrf/JSON_QUERY.html#GUID-6D396EC4-D2AA-43D2-8F5D-08D646A4A2D9)] nor is it relating to some industry standard like [JMESPath](https://jmespath.org/).

However, when we look at the inefficient use of `JSON_QUERY` above, we see `@myJson` being parsed _twice_. We may assume that the makers of SQL Server would not want to encourage these patterns. I daresay that `OPEN_JSON` table-valued function allows us to parse `@myJson` _once_.

## by default `OPEN_JSON` returns three columns

Sans the very important `WITH` clause [📖 [docs](https://learn.microsoft.com/en-us/sql/t-sql/functions/openjson-transact-sql?view=sql-server-ver16#with_clause-1)], `OPEN_JSON` returns three columns:

In [3]:
DECLARE @myJson NVARCHAR(MAX) = CONVERT(NVARCHAR(MAX), SESSION_CONTEXT(N'myJson'))

SELECT * FROM OPENJSON(@myJson)

(1 row affected)

Total execution time: 00:00:00.004

key,value,type
Orders,"[ { ""OrderNumber"":""SO43659"", ""OrderDate"":""2011-05-31T00:00:00"", ""AccountNumber"":""AW29825"", ""ItemPrice"":2024.9940, ""ItemQuantity"":1 }, { ""OrderNumber"":""SO43661"", ""OrderDate"":""2011-06-01T00:00:00"", ""AccountNumber"":""AW73565"", ""ItemPrice"":2024.9940, ""ItemQuantity"":3 } ]",4


>By default, the `OPENJSON` table-valued function returns three columns, which contain the key name, the value, and the type of each `{key:value}` pair found in `jsonExpression`.

As usual in the misery of software development, we have more than one level of disappointment. Not only are we not getting the table columns we are looking for, but also we have only _one_ row returned when we know we have _two_ orders 😐

For the first time, in this extremely pretentious document, we see shall see `OPENJSON` working together with `JSON_QUERY`:

In [4]:
DECLARE @myJson NVARCHAR(MAX) = CONVERT(NVARCHAR(MAX), SESSION_CONTEXT(N'myJson'))

SELECT * FROM OPENJSON(JSON_QUERY(@myJson, '$.Orders'))

(2 rows affected)

Total execution time: 00:00:00.005

key,value,type
0,"{ ""OrderNumber"":""SO43659"", ""OrderDate"":""2011-05-31T00:00:00"", ""AccountNumber"":""AW29825"", ""ItemPrice"":2024.9940, ""ItemQuantity"":1 }",5
1,"{ ""OrderNumber"":""SO43661"", ""OrderDate"":""2011-06-01T00:00:00"", ""AccountNumber"":""AW73565"", ""ItemPrice"":2024.9940, ""ItemQuantity"":3 }",5


By nesting `JSON_QUERY` in `OPENJSON`, we can target the `Orders` array and project the two orders we expect. Also notice that the types have changed to `5` (object) from the previous type of `4` (array) [📖 [docs](https://learn.microsoft.com/en-us/sql/t-sql/functions/openjson-transact-sql?view=sql-server-ver16#return-value)] .

## project domain data from `OPENJSON` with the `WITH` clause

We are about to overcome all of the disappointment mentioned in this writ once we make proper use of the aforementioned `WITH` clause [📖 [docs](https://learn.microsoft.com/en-us/sql/t-sql/functions/openjson-transact-sql?view=sql-server-ver16#with_clause-1)]:

In [5]:
DECLARE @myJson NVARCHAR(MAX) = CONVERT(NVARCHAR(MAX), SESSION_CONTEXT(N'myJson'))

SELECT
    *
FROM
    OPENJSON(JSON_QUERY(@myJson, '$.Orders'))
    WITH (
            ORDER_NUMBER   NVARCHAR(32) '$.OrderNumber'
        ,   ORDER_DATE     DATETIME     '$.OrderDate'
        ,   ACCOUNT_NUMBER NVARCHAR(32) '$.AccountNumber'
        ,   PRICE          DECIMAL      '$.ItemPrice'
        ,   QUANTITY       INT          '$.ItemQuantity'
    )


(2 rows affected)

Total execution time: 00:00:00.003

ORDER_NUMBER,ORDER_DATE,ACCOUNT_NUMBER,PRICE,QUANTITY
SO43659,2011-05-31 00:00:00.000,AW29825,2025,1
SO43661,2011-06-01 00:00:00.000,AW73565,2025,3


## <!-- -->

🐙🐱[BryanWilhite](https://github.com/BryanWilhite)